In [1]:
## Imports

import numpy as np 
import math
import time
from math import exp, log, pi, sqrt
from scipy import stats
import yahoo_fin
from yahoo_fin import options
from yahoo_fin import stock_info
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime
import pandas as pd

## Position Analysis Function

In [46]:
class PositionAnalysis:

    def __init__(self, portfolio, j):
        self.portfolio = portfolio      # Dataframe containing all positions
        self.j = j                      # Used for selecting position number

#================================ CALLS ====================================================================

    def pos_delta(self, portfolio, j):           # Return the net delta of a particular position

        delta = portfolio[j][1] * portfolio[j][2]

        return delta

    def port_delta(self, portfolio):         # Return the net delta of the portfolio

        portdelta = 0
        for i in range(1, len(portfolio)):
            delta_i = portfolio[i][1] * portfolio[i][2]
            portdelta = portdelta + delta_i

        return portdelta
            

## Option Pricing / Greeks Calculator Function

In [7]:
class OptionsPricing:

    def __init__(self, S, K, T, r, q, sigma):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q 
        self.sigma = sigma 

#================================ CALLS ====================================================================

    def call_price(self):           # Return the theoretical value of the call option

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        price = S * np.exp(-q * T) * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)

        return price

    def call_delta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        delta = stats.norm.cdf(d1) * exp(-q * T)

        return delta

    def call_gamma(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma

    def call_theta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        theta = (-(S * exp(-1 * (d1 ** 2) / 2) / sqrt(2 * pi) * sigma * exp(-q * T)/ (2 * sqrt(T))) - (r * (K * exp(-r * T)) * stats.norm.cdf(d2)) + (q * S * stats.norm.cdf(d1) * exp(-q * T))) / 365      #Theta defined as the loss of value every calendar day (not trading day)

        return theta

    def call_vega(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        vega = exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * exp(-q * T) * S * sqrt(T) / 100

        return vega
    
    def call_rho(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        rho = K * T * exp(-r * T) * stats.norm.cdf(d2) / 100  

        return rho


#================================ PUTS ====================================================================

    def put_price(self):            # Return the theoretical value of the put option

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        price = K * np.exp(-r * T) * stats.norm.cdf(-d2) - S * np.exp(-q * T) * stats.norm.cdf(-d1)

        return price

    def put_delta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        delta = (stats.norm.cdf(d1)-1) * exp(-q * T)

        return delta

    def put_gamma(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma

    def put_theta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        theta = (-(S * exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * sigma * exp(-q * T) / (2 * sqrt(T))) + (r * K * exp(-r * T) * stats.norm.cdf(-d2)) - (q * S * stats.norm.cdf(-d1) * exp(-q * T))) / 365         # Theta defined as the loss of value every calendar day (365), not every trading day

        return theta

    def put_vega(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        vega = exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * exp(-q * T) * S * sqrt(T) / 100

        return vega

    def put_rho(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        rho = -K * T * exp(-r * T) * stats.norm.cdf(-d2) / 100

        return rho


## Portfolio Components

In [55]:
cash      = ['$USD', 137000, +1]         # Cash position
position1 = ['CAVA', 1400, +1] 
position2 = ['AMD', 300, +1]
position3 = ['CCJ', 1000, +1]
position4 = ['CCJ', 1000, +1]
position5 = ['NVDA', 82, +1]
position6 = ['SMCI', 100, +1]
position7 = ['SNOW', 300, +1]
position8 = ['PATH', 2000, +1]
position9 = ['PRIM', 1200, +1]
position10 = ['DDOG', 450, +1]
position11 = ['ONON', 1050, +1]
position12 = ['FLR', 1000, +1]
position13 = ['SG', 2200, +1]

portfolio = [cash, position1, position2, position3, position4, position5, position6, position7, position8, position9, position10, position11, position12, position13]

data = pd.DataFrame(portfolio, columns= ['Instrument', 'Position', 'Direction'], index= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])   # index should go up to the number of positions held (13)


nav = cash[1] 
for i in range(1,len(portfolio)):
    price = stock_info.get_live_price(f"{portfolio[i][0]}")           # Return current price of position i
    i_value = price * portfolio[i][1]                                 # value = price * position size
    nav = nav + i_value                                               # append nav with value of position i

print(data)
print(f"\nNet Asset Value: {nav}\n")


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ret

   Instrument  Position  Direction
0        $USD    137000          1
1        CAVA      1400          1
2         AMD       300          1
3         CCJ      1000          1
4         CCJ      1000          1
5        NVDA        82          1
6        SMCI       100          1
7        SNOW       300          1
8        PATH      2000          1
9        PRIM      1200          1
10       DDOG       450          1
11       ONON      1050          1
12        FLR      1000          1
13         SG      2200          1

Net Asset Value: 678432.737449646



/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


In [57]:
pos_number = 1      # Input position number here to calculate delta

pos_delta = PositionAnalysis(portfolio, j)

delta_j = pos_delta.pos_delta(portfolio, pos_number)

port_delta = PositionAnalysis(portfolio, j)

total_delta = port_delta.port_delta(portfolio)          # Calculate delta for the entire portfolio

print(f"\nDelta of position {pos_number}: {delta_j}\n")
print(f"Portfolio Delta: {total_delta}")


Delta of position 1: 1400

Portfolio Delta: 12082


## Input Metrics of the Option

In [65]:
pos_number = 1      # Input position number to identify the ticker to price options for 

instrument = portfolio[pos_number][0]
price = stock_info.get_live_price(f"{instrument}")           # Underlying price
S = round(price, 2)
print(f"\nCurrent Price of {instrument}: {S}\n")


Current Price of CAVA: 38.87



/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


In [73]:
if portfolio[pos_number][2] > 0:        # If the position is LONG
        option = 'PUT'
        chain = options.get_puts(f"{instrument}")         # Buy Puts
        print(f"{instrument} Put Strike Prices\n\n", chain['Strike'])
else: 
        option = 'CALL'
        chain = options.get_calls(f"{instrument}")        # Buy Calls
        print(f"{instrument} Call Strike Prices\n\n", chain['Strike'])

CAVA Put Strike Prices

 0     17.5
1     20.0
2     21.0
3     22.0
4     22.5
5     23.0
6     24.0
7     24.5
8     25.0
9     26.0
10    26.5
11    27.0
12    27.5
13    28.0
14    28.5
15    29.0
16    29.5
17    30.0
18    30.5
19    31.0
20    31.5
21    32.0
22    32.5
23    33.0
24    33.5
25    34.0
26    34.5
27    35.0
28    35.5
29    36.0
30    37.0
31    38.0
32    39.0
33    40.0
34    42.0
35    43.0
36    44.0
37    45.0
38    50.0
39    55.0
40    60.0
41    65.0
42    70.0
43    75.0
44    80.0
45    85.0
Name: Strike, dtype: float64


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)


In [74]:
K = float(input("Select a Strike Price: "))     # Strike Price

print(f"\nSelected {instrument} {option} Strike Price: {K}\n")


Selected CAVA PUT Strike Price: 43.0



In [77]:
dates = options.get_expiration_dates(f"{ticker}")
print(f"\nSelect a {option} Expiry Date:\n\n", dates)


Select a PUT Expiry Date:

 ['December 15, 2023', 'December 22, 2023', 'December 29, 2023', 'January 5, 2024', 'January 12, 2024', 'January 19, 2024', 'January 26, 2024', 'February 16, 2024', 'March 15, 2024', 'April 19, 2024', 'June 21, 2024', 'July 19, 2024', 'September 20, 2024', 'December 20, 2024', 'January 17, 2025', 'June 20, 2025', 'September 19, 2025', 'December 19, 2025', 'January 16, 2026']


In [78]:
date1 = input("\nExpiry Date (MM/DD/YY): ")
date_str = date1
date_object = datetime.strptime(date_str, '%m/%d/%y').date()
    #print(type(date_object))
    #print(date_object)

from datetime import date
today = date.today()
td = date_object - today
print(f"\nExpiry Date: {date1}")
print(f"\nDays to Expiry: {td.days}")

T = td.days / 365



Expiry Date: 01/12/24

Days to Expiry: 31


In [85]:
r = stock_info.get_live_price("^IRX")/100
rate = round(r, 4)
print(f"Risk-Free Rate: {rate*100}%\n")

Risk-Free Rate: 5.25%



/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


In [91]:
div = stock_info.get_dividends(instrument, "01-01-2023")

if len(div['dividend']) == 0:
    q = 0
    print(f"{q}\n%")
    
else:
    i = len(div)
    q = i * div['dividend'][i-1] / S
    print(f"Current Dividend Yield: {round(q, 2)}\n")                                                   # annual dividend yield


KeyError: 'dividend'

In [15]:
from dateutil.relativedelta import relativedelta
minusoneyear = today - relativedelta(years=1)

df = stock_info.get_data(f"{ticker}", start_date= minusoneyear, end_date= today)
df['close']

log_returns = np.log(df.close/df.close.shift(1)).dropna()
daily_std = log_returns.std()
annualized_vol = daily_std * np.sqrt(252)
sigma = round(annualized_vol, 4)
sigma 

0.2167

In [16]:
model = OptionsPricing(S, K, T, r, q, sigma)

callprice = model.call_price()
calldelta = model.call_delta()
callgamma = model.call_gamma()
calltheta = model.call_theta()
callvega = model.call_vega()
callrho = model.call_rho()

putprice = model.put_price()
putdelta = model.put_delta()
putgamma = model.call_gamma()
puttheta = model.put_theta()
putvega = model.put_vega()
putrho = model.put_rho()


In [17]:
callprice

0.06617011658877914